## Problem Statement : Hospital Patient Data Analysis
### Context:
A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one with patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups.


#### 1.	Load the patient dataset and show summary with info().

In [68]:
import pandas as pd 
patients = pd.read_csv('Patient_Data.csv')

print(patients.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes
None


#### 2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].


In [69]:
patients_cleaned = patients[['PatientID','Department','Doctor','BillAmount']]

patients_cleaned

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN
5,101,Cardiology,Dr. Smith,5000.0


#### 3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].

In [70]:
patients_droped = patients.drop(['ReceptionistID', 'CheckInTime'], axis=1)
patients_droped.head()

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


#### 4.	Use groupby to find total bill amount per department.

In [71]:
dept_total = patients_droped.groupby('Department')['BillAmount'].sum().reset_index()
dept_total

,Department,BillAmount
0,Cardiology,16200.0
1,Dermatology,0.0
2,Neurology,0.0
3,Orthopedics,7500.0


#### 5.	Remove duplicate patient records based on PatientID.

In [72]:
patients_final = patients_droped.drop_duplicates(subset='PatientID', keep='first')
patients_final

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


#### 6.	Fill missing BillAmount values with the mean bill amount.

In [67]:
mean_bill = patients_final['BillAmount'].mean()
patients_final['BillAmount'].fillna(mean_bill)
patients_final

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000
1,102,Bob,Neurology,Dr. John,6233.333333
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000
3,104,David,Cardiology,Dr. Smith,6200.000000
4,105,Eva,Dermatology,Dr. Rose,6233.333333


#### 7.	Merge the billing dataset with patient dataset on PatientID.

In [64]:
billing_df = pd.read_csv('Billing_Data.csv')
merged_df = pd.merge(patients_final, billing_df, on='PatientID', how='inner')
merged_df

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000,3000
1,102,Bob,Neurology,Dr. John,6233.333333,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.000000,3000,3200
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000,4000


#### 8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).

In [65]:
new_patients = pd.DataFrame({
    'PatientID': [106, 107],
    'Name': ['Frank', 'Grace'],
    'Department': ['Neurology', 'Cardiology'],
    'Doctor': ['Dr. John', 'Dr. Smith'],
    'BillAmount': [5800, 7200],
    'InsuranceCovered': [1000, 2000],
    'FinalAmount': [4800, 7200]
})
merged_df = pd.concat([merged_df, new_patients], ignore_index=True)
merged_df

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000,3000
1,102,Bob,Neurology,Dr. John,6233.333333,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.000000,3000,3200
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000,4000
5,106,Frank,Neurology,Dr. John,5800.000000,1000,4800
6,107,Grace,Cardiology,Dr. Smith,7200.000000,2000,7200


#### 9.	Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).

In [66]:
new_billing_cols = pd.DataFrame({
    'PaymentStatus': ['Paid', 'Pending', 'Paid', 'Paid', 'Pending', 'Paid', 'Paid'],
    'PaymentMethod': ['Card', 'Cash', 'Insurance', 'Card', 'Cash', 'Card', 'Insurance']
})
final_df = pd.concat([merged_df, new_billing_cols], axis=1)
final_df

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,PaymentStatus,PaymentMethod
0,101,Alice,Cardiology,Dr. Smith,5000.000000,2000,3000,Paid,Card
1,102,Bob,Neurology,Dr. John,6233.333333,1500,3500,Pending,Cash
2,103,Charlie,Orthopedics,Dr. Lee,7500.000000,2500,5000,Paid,Insurance
3,104,David,Cardiology,Dr. Smith,6200.000000,3000,3200,Paid,Card
4,105,Eva,Dermatology,Dr. Rose,6233.333333,1000,4000,Pending,Cash
5,106,Frank,Neurology,Dr. John,5800.000000,1000,4800,Paid,Card
6,107,Grace,Cardiology,Dr. Smith,7200.000000,2000,7200,Paid,Insurance
